In [33]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools

In [22]:
def read_fasta_file(path):
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh=open(m6a_benchmark_dataset)
    seq=[]
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\r\n',''))
    fh.close()
    matrix_data=np.array([list(e) for e in seq])
    return matrix_data

In [27]:
def AthMethPre_extract_one_line(data_line):
    '''
    extract features from one line, such as one m6A sample
    '''
    A=[0,0,0,1]
    U=[0,0,1,0]
    C=[0,1,0,0]
    G=[1,0,0,0]
    N=[0,0,0,0]
    feature_representation={"A":A,"U":U,"C":C,"G":G,"N":N}
    beginning=0
    end=len(data_line)-1
    one_line_feature=[]
    alphabet='AUCG'
    matrix_three=["".join(e) for e in itertools.product(alphabet, repeat=3)]# AAA AAU AAC ...
    feature_three=np.zeros(64)
    A=0.1260
    U=0.1335
    C=0.1340
    G=0.0806
    AUCG=[sum(e) for e in itertools.product([A,U,C,G], repeat=3)]# AAA AAU AAC ...
    for index,data in enumerate(data_line):
        if "".join(data_line[index:(index+3)]) in matrix_three and index <= end-2:
            feature_three[matrix_three.index("".join(data_line[index:(index+3)]))]+=1     
    sum_three=np.sum(feature_three)
    feature_three=feature_three/sum_three
    feature_three=feature_three*AUCG
    one_line_feature.extend(feature_three)
    return one_line_feature


In [28]:
def AthMethPre_feature_extraction(matrix_data):
    final_feature_matrix=[AthMethPre_extract_one_line(e) for e in matrix_data]
    return final_feature_matrix

In [29]:

m6a_benchmark_dataset='/home02/chenhuangrong/m6a_data.txt'
matrix_data=read_fasta_file(m6a_benchmark_dataset)
final_feature_matrix=AthMethPre_feature_extraction(matrix_data)
pd.DataFrame(final_feature_matrix).to_csv('/home02/chenhuangrong/PseEIIP.csv',header=None,index=False)




In [30]:
def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError

In [31]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [40]:
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0) #above was used to generate xlsx format Excel file
RNA_code='ACGU' 
interval=3 # RNA_code and interval used mix used to generate AAA AAC AAG ...
division_num=10
divided_num=np.float(division_num)# ten fold so the result should divided by ten


# seq=[e.replace('U','A') for e in seq]
# seq=[e.replace('G','C') for e in seq]
seq=pd.read_csv('/home02/chenhuangrong/PseEIIP.csv',header=None,index_col=False)
seq=seq.values

positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]



#define variable to save data which will be saved later --- begining
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0
#define variable to save data which will be saved later --- end


#shuffle the data of positive and negative begining
kf = KFold(n_splits=division_num,shuffle=True)  # positive and negative samples will be shuffled 
for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
#shuffle the data of positive and negative end


    #generate train and test data begining
    X_train = np.array(positive_negative_x_train)
    Y_train = list(map(lambda x: 1, xrange(len(X_train) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(X_train) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(positive_negative_y_train)
    Y_test  = list(map(lambda x: 1, xrange(len(X_test) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(X_test) / 2)))
    Y_test.extend(Y2_test )
    Y_test = np.array(Y_test)
    #generate train and test data end

    
    # training model and optimized parameters of C and gamma begining
    svc = svm.SVC(probability=True)
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    # training model and optimized parameters of C and gamma end
    
    
    #print best C and gamma begining
    C=clf.best_params_['C']
    gamma=clf.best_params_['gamma']
    print 'c:',C,'gamma:',gamma
    #print best C and gamma end
    
    
    #getting predict probability and predict label begining
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    #getting predict probability and predict label begining

    #the process of generating  usefule data begining
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)

    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
    #the process of generating  usefule data end
    
#the process of save  data to disk begining   
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
# pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/PseEIIP%d.csv'%select_num,header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[str(X_train.shape[1]),"正："+str(pos_all)+'负：'+str(neg_all),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all]
final_out_to_excel.append(savedata)

print savedata

pd.DataFrame(final_out_to_excel).to_excel("/home02/chenhuangrong/PseEIIP.xlsx",sheet_name="independent_test",index=False,header=False)
#the process of save  data to disk end

c: 32.0 gamma: 1.78179743628
0.671755725191
ACC_all: 0.671755725191
c: 14.2543794902 gamma: 4.0
0.702290076336
ACC_all: 1.37404580153
c: 6.34960420787 gamma: 1.78179743628
0.610687022901
ACC_all: 1.98473282443
c: 32.0 gamma: 0.353553390593
0.679389312977
ACC_all: 2.6641221374
c: 6.34960420787 gamma: 4.0
0.637404580153
ACC_all: 3.30152671756
c: 14.2543794902 gamma: 4.0
0.751908396947
ACC_all: 4.0534351145
c: 14.2543794902 gamma: 4.0
0.652671755725
ACC_all: 4.70610687023
c: 32.0 gamma: 4.0
0.646153846154
ACC_all: 5.35226071638
c: 32.0 gamma: 1.78179743628
0.638461538462
ACC_all: 5.99072225484
c: 32.0 gamma: 0.353553390593
0.657692307692
ACC_all: 6.64841456254
['64', '\xe6\xad\xa3\xef\xbc\x9a1307.0\xe8\xb4\x9f\xef\xbc\x9a1307.0', 'svmC:32.0gamma:0.353553390593', 0.66484145625367008, 0.6558019979433161, 0.6977334116265415, 0.6977334116265415, 0.6319495008807985, 0.6631482513695881, 0.67547861587488223, 0.67547861587488223, 0.3310442722190674, 0.72664582271085776, 912.0, 395.0, 481.0, 826.0